In [53]:
import pandas as pd
import time
from nba_api.stats.endpoints import leaguedashplayerstats
from rapidfuzz import process, fuzz

In [54]:
# Step 1: Load Google Sheet
sheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTvnvbpz59twmXa0gIXKpB1W7dHiiJu4Jd-9j5Ftau-Y4_QR_xN0aiMfwI2cuenXC0E5r7xZPlLhGOz/pub?gid=0&single=true&output=csv"
top100_df = pd.read_csv(sheet_url)
top100_df = top100_df[['Player', 'Rank', 'OVR']]
top100_df['Player_clean'] = top100_df['Player'].str.strip().str.lower()

In [55]:
# Step 2: Get NBA stats
print("Fetching NBA stats...")
nba_stats = leaguedashplayerstats.LeagueDashPlayerStats(
    season='2024-25',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base'
)
df_nba = nba_stats.get_data_frames()[0]

# Calculate per-game stats
df_nba['PPG'] = df_nba['PTS'] / df_nba['GP']
df_nba['RPG'] = df_nba['REB'] / df_nba['GP']
df_nba['APG'] = df_nba['AST'] / df_nba['GP']

# Efficiency metrics
df_nba['FG%'] = df_nba['FG_PCT']
df_nba['3P%'] = df_nba['FG3_PCT']
df_nba['FT%'] = df_nba['FT_PCT']
df_nba['TS%'] = df_nba['PTS'] / (2 * (df_nba['FGA'] + 0.44 * df_nba['FTA']))

# Role metrics
df_nba['MPG'] = df_nba['MIN'] / df_nba['GP']
df_nba['USG%'] = df_nba['USG_PCT'] if 'USG_PCT' in df_nba.columns else None
df_nba['Starter'] = None  # Placeholder: GS not available from this endpoint

# Defensive stats
df_nba['SPG'] = df_nba['STL'] / df_nba['GP']
df_nba['BPG'] = df_nba['BLK'] / df_nba['GP']
df_nba['TPG'] = df_nba['TOV'] / df_nba['GP']

# Advanced metrics placeholders (if not available)
for col in ['OFF_RATING', 'DEF_RATING', 'OBPM', 'DBPM']:
    if col not in df_nba.columns:
        df_nba[col] = None

df_nba['Player_clean_api'] = df_nba['PLAYER_NAME'].str.strip().str.lower()


Fetching NBA stats...


In [56]:
# Step 3: Scrape Basketball Reference stats
print("Scraping advanced stats from Basketball Reference...")
time.sleep(2)
url = 'https://www.basketball-reference.com/leagues/NBA_2025_advanced.html'
tables = pd.read_html(url)
df_adv = tables[0]
df_adv = df_adv[~df_adv['Player'].str.contains('Player')]
df_adv = df_adv.drop_duplicates(subset='Player')
df_adv['Player_clean_br'] = df_adv['Player'].str.strip().str.lower()

adv_cols = ['Player_clean_br', 'PER', 'WS', 'BPM', 'OBPM', 'DBPM']
df_adv = df_adv[adv_cols]
for col in ['PER', 'WS', 'BPM', 'OBPM', 'DBPM']:
    df_adv[col] = pd.to_numeric(df_adv[col], errors='coerce')

Scraping advanced stats from Basketball Reference...


In [57]:
# Step 4: Fuzzy match Google Sheet players to NBA API
def fuzzy_match(player, choices):
    match, score, _ = process.extractOne(player, choices, scorer=fuzz.WRatio)
    return match if score > 80 else None  # You can adjust threshold

# Match to nba_api
top100_df['match_api'] = top100_df['Player_clean'].apply(
    lambda x: fuzzy_match(x, df_nba['Player_clean_api'].tolist())
)

# Match to Basketball Reference
top100_df['match_br'] = top100_df['Player_clean'].apply(
    lambda x: fuzzy_match(x, df_adv['Player_clean_br'].tolist())
)

In [60]:
# Step 5: Merge based on fuzzy-matched names
df_nba_matched = df_nba.rename(columns={'Player_clean_api': 'match_api'})
df_adv_matched = df_adv.rename(columns={'Player_clean_br': 'match_br'})

merged = pd.merge(top100_df, df_nba_matched, on='match_api', how='left')
merged = pd.merge(merged, df_adv_matched, on='match_br', how='left')

# Finalize columns
final_df = merged[['Rank', 'Player', 'OVR', 'PER', 'WS', 'BPM', 'PPG', 'RPG', 'APG', 'FG%', '3P%', 'FT%', 'TS%', 'MPG', 'SPG', 'BPG', 'TPG']]
final_df = final_df.sort_values(by='Rank').reset_index(drop=True)

# Save to CSV
final_df.to_csv("top100_nba_2025_ranked_stats.csv", index=False)
print("✅ Final dataset saved to 'top100_nba_2025_ranked_stats.csv'")

✅ Final dataset saved to 'top100_nba_2025_ranked_stats.csv'
